In [117]:
import sqlite3
import pandas as pd

# Step 1: Connect to DB
conn = sqlite3.connect("chinook.db")

# Step 2: Execute query and send results to pandas dataframe:
df = pd.read_sql_query("""          
                    SELECT 
                        ca.artist_name,
                        ca.year,
                        SUM(ca.UnitPrice) total,
                        rank() over (partition by year order by SUM(ca.UnitPrice) desc) rank
                    FROM
                        (SELECT 
                                invoice_items.TrackId,
                                invoice_items.UnitPrice,
                                invoices.Total,
                                invoices.InvoiceId AS invoice_id,
                                STRFTIME( '%Y', invoices.InvoiceDate) AS year,
                                invoices.BillingState,
                                tracks.AlbumID AS album_id,
                                artists.ArtistID AS artist_id,
                                artists.Name AS artist_name
                        FROM
                            invoice_items
                        INNER JOIN invoices ON invoices.InvoiceId = invoice_items.InvoiceId
                        INNER JOIN tracks ON tracks.TrackId = invoice_items.TrackId
                        INNER JOIN albums ON albums.AlbumId = tracks.AlbumId
                        INNER JOIN artists ON artists.ArtistId = albums.ArtistId
                        WHERE
                            invoices.BillingState = 'CA') ca
                    GROUP BY ca.artist_id , 2;                    
                 """
              ,conn)
top3_CA = df[df['rank'] <= 3]
print(top3_CA)



              artist_name  year  total  rank
0                The Cult  2009   4.95     1
1             Deep Purple  2009   3.96     2
2                  Djavan  2009   2.97     3
3            Eric Clapton  2009   2.97     3
4               The Clash  2009   2.97     3
11            Iron Maiden  2010  13.86     1
12         Caetano Veloso  2010   1.98     2
13        Various Artists  2010   1.98     2
14               Ed Motta  2010   1.98     2
15            Deep Purple  2010   1.98     2
16            Miles Davis  2010   1.98     2
17             Gene Krupa  2010   1.98     2
18    Toquinho & Vinícius  2010   1.98     2
19           Eric Clapton  2010   1.98     2
22              Van Halen  2011   5.94     1
23  Red Hot Chili Peppers  2011   4.95     2
24    Toquinho & Vinícius  2011   2.97     3
25            Raul Seixas  2011   2.97     3
31             The Office  2012   5.97     1
32   Antônio Carlos Jobim  2012   5.94     2
33            Iron Maiden  2012   5.94     2
34        